advanced-boto3

In [2]:
import boto3
#import pandas as pd
import os
import zipfile

In [3]:
#start a session
session = boto3.Session(profile_name='default',region_name='us-west-2')
ec2_client = session.client('ec2')

In [4]:
# Define a Launch Template
def create_launch_template():
    securityGroupIds=['sg-044f4a8ac6a19c5bf'],  # Replace with your security group ID
    keyname='vockey'  # Replace with your key pair name
    ec2_client.create_launch_template(
        LaunchTemplateName='MyLaunchTemplate',
        VersionDescription='v1',
        LaunchTemplateData={
            'InstanceType': 't2.micro',
            'ImageId': 'ami-0c55b159cbfafe1f0',  # Amazon Linux 2 AMI
            'KeyName': keyname, 
            'SecurityGroupIds': securityGroupIds,  
            'UserData': '''#!/bin/bash
                            yum update -y
                            yum install -y httpd
                            systemctl start httpd
                            systemctl enable httpd'''
        }
    )

In [ ]:
#Request a Spot Fleet
def request_spot_fleet():
    response = ec2_client.request_spot_fleet(
        SpotFleetRequestConfig={
            'TargetCapacity': 1,
            'LaunchTemplateConfigs': [
                {
                    'LaunchTemplateSpecification': {
                        'LaunchTemplateName': 'MyLaunchTemplate',
                        'Version': '$Latest'
                    },
                    'Overrides': [
                        {
                            'InstanceType': 't2.micro',
                            'SubnetId': 'subnet-0bb1c79de3EXAMPLE'  # Replace with your subnet ID
                        }
                    ]
                }
            ],
            'IamFleetRole': 'arn:aws:iam::123456789012:role/aws-ec2-spot-fleet-role',  # Replace with your IAM role ARN
            'SpotPrice': '0.01',  # Set your maximum price per hour
            'Type': 'request'
        }
    )
    print("Spot Fleet Request ID:", response['SpotFleetRequestId'])